<img src='../images/gdd-logo.png' width='300px' align='right' style="padding: 15px">

# <mark style='background-color:white;color:#364069'>Object Oriented Programming</mark>

Now that you have a `Deck` of cards, this notebook will demonstrate some best practice and more advanced ways of working with Object Oriented Programming within Python.

In particular this notebook covers:

- [Recap of the `Deck` of cards](#recap)


- [Class and Instance variables: Whether to include attributes in the __init__ function](#class-instance)
    
    
- [Dunder methods](#dunder)
    - [Using the in-built `len()` function](#len)
    - [`__str__`](#str)
    - [<mark>Exercise - Add dunder methods</mark>](#ex-dunder)


- [Better integration](#better)
    - [Leading underscore](#leading)
    - [Protecting variables further](#protected)


- [Conclusion](#conclusion)

In [ ]:
import collections

---
<a id='deck'></a>
# Recap of the `Deck` of cards

In the previous notebook you implemented a class called `Deck`, which represents a (French) card deck. 

<img src='../images/french-card.jpeg'>

In [ ]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

class Deck:
    ranks = '23456789TJQKA'
    suits = '♠♥♦♣'
    
    def __init__(self):
        self.cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]
        self.dealt_cards = []
    
    def deal(self):
        dealt_card = self.cards.pop()
        self.dealt_cards.append(dealt_card)
        return dealt_card
    
    def size(self):
        return len(self.cards)
    
    def check_ace(self):
        return self.cards[-1].rank == 'A'
    
deck = Deck()
deck.deal()
deck.dealt_cards

---

<a id='class-instance'></a>
## Class and Instance variables: Whether to include attributes in the `__init__` function

It is also possible to define variables outside the `__init__` method which will equally be accessible (and mutable) using the `class.attribute` syntax. However, there is a slight difference:

- variables defined **outside** the `__init__` method belong to the class (*class variables*),
- while variables defined **inside** the `__init__` method belong to an instance of a class (*instance variables*).

Luckily, in most cases, this difference often does not matter and your code will probably work both ways.

An in-depth discussion can be found [here](https://www.atatus.com/blog/class-variables-vs-instance-variables-in-java/), but for now below it the difference is illustrated on a simple example:

In [ ]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

class DeckClassVars:
    
    # define as class variables
    ranks = 'A23456789TJQK'
    suits = '♠♥♦♣'
      
    def __init__(self):
        self.cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]

Notice that without instantiating the class above, you can already return the `ranks` (or `suits`) attribute:

In [ ]:
DeckClassVars.ranks

This because the attributes `ranks` and `suits` belong to the class itself.

Accordingly, examine what happens when we do make an instance of this class, and then update the class itself.

In [ ]:
# make an instance of the class
deck_with_class_vars = DeckClassVars()

deck_with_class_vars.suits

In [ ]:
# update the class
DeckClassVars.suits = ['Water', 'Air', 'Fire', 'Earth']

deck_with_class_vars.suits

In contrast, take a look at the class below, where `ranks` and `attributes` are part of the `__int__` method:

In [ ]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

class DeckInstanceVars:
      
    def __init__(self):
        # define as instance variables
        self.ranks = 'A23456789TJQK'
        self.suits = '♠♥♦♣'
    
        self.cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]

The class itself does not have a `ranks` (or `suits`) attribute. Uncomment and run the cell below to verify this.

In [ ]:
# DeckInstanceVars.ranks

Examine what happens when we make an instance of this class, and then update the class itself.

In [ ]:
deck_with_instance_vars = DeckInstanceVars()

deck_with_instance_vars.suits

In [ ]:
DeckInstanceVars.suits = ['Water', 'Air', 'Fire', 'Earth']

deck_with_instance_vars.suits

### Bottom line 
There is a difference between **class** variables and **instance** variables. Often, this difference is not noticable, but it can lead to problems when object attributes are changed with a method or (like in our example) via direct re-assignment.

> <mark>**Best practice:**</mark> Define all variables that could differ from instance to instance (think of the `.shape` and `.dtypes` of a `pd.DataFrame`) into the `__init__` method, and define static class variables only if they are shared across all instances of a class (for example, the `pd.DataFrame` class should always only have 2 dimensions).

---
<a id='dunder'></a>
## Dunder methods

With the Deck class we made earlier, we can use the `size` method to find out how many cards there are in the deck.

In [ ]:
deck = Deck()
deck.size()

However, there is a more "pythonic" way of finding the size of the deck. 

Currently, other programmers - including the future us - who use your class need to remember the name of the method to return the number of cards. 

To save mental effort, you are now going to implement a dunder method, that gets invoked when you use an already existing function.

<a id='len'></a>

### Add the dunder method `__len__`

How to get the number of items in a list

In [ ]:
my_list = [1, 2, 3, 4]
len(my_list)

Would it be possible to do the same with Deck?

In [ ]:
deck = Deck()
# len(deck)

The function `len` works by calling the (dunder) method `__len__` of an object.

Let's change the method for counting cards to opearte like this.

In [ ]:
class Deck:
    ranks = '23456789TJQKA'
    suits = '♠♥♦♣'
    
    def __init__(self):
        self.cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]
    
    def __len__(self):
        return len(self.cards)
    
    def deal(self):
        return self.cards.pop()

Now that you have a dunder method `__len__` this function gets called when you apply `len` to the instance of `Deck`.

Let's try this out...

In [ ]:
deck = Deck()

print(len(deck))

for _ in range(10): 
    deck.deal()

print(len(deck))

After we played a couple cards, we see the length has decreased.

Great! Now we have an object that interacts like other python objects. We would like to have more of these tricks!

<a id='str'></a>
### Dunder method `__str__`

Notice the print. It contains the class name, with module `__main__` and the hexidecimal id

In [ ]:
print(deck)

While Python can interpret this, it is not a very useful object representation for a user. 

However, we can use the dunder (double-underscore) method `__str__` to control what will be printed.


> To be a little more precise: `__str__` is called by the `str()` built-in function and implicitly used by the `print()` function. It should return a string suitable for display to end users. See python docs [here for `print`](https://docs.python.org/3/library/functions.html#print) and [here for `str`](https://docs.python.org/3/library/stdtypes.html#str).

<a id='ex-dunder'></a>
### <mark>Exercise - Add a dunder</mark>

1. Implement a `method` such that when you called `print(deck)` it prints the string `Deck(suits=♠♥♦♣, ranks=23456789TJQKA)`

2. Look up the dunder method [`__getitem__`](https://docs.python.org/3/reference/datamodel.html#object.__getitem__):

    - Implement it for your deck so that you can run `deck[0]` to retrieve the first card in the deck. 

    - Try out other ways of slicing the deck (eg. select the first 5 cards)

3. From the link above, find a dunder method that will be implicitly invoked to set a value to an item of a list. I.e. we should be able to execute the code:
```python
deck[0] = Card('A', '♦')
```

**Bonus**: Import the function `random.shuffle` and execute it on the deck of cards. What happens to `deck._cards`?

**Answers**: Uncomment and run the following to see solutions

In [ ]:
# %load ../answers/ex-dunder-1.py

In [ ]:
# %load ../answers/ex-dunder-2.py

In [ ]:
# %load ../answers/ex-dunder-3.py

### A note on `__getitem__` and `__setitem__`

These two dunder methods are the harder ones to understand the implementation when thinking about your card deck. 

- `__getitem__` is a method used for getting the value of an item
- `__setitem__` is a method used for assigning a value to an item

More information and examples are covered in the [getitem_and_setitem]() notebook.

### Further dunder methods

You are getting closer and closer to a "pythonic" card deck! In addition to the ones you've seen already, there are many more interesting dunder methods such as:

- `__iter__` and `__next__`
- `__repr__`
- `__add__`, `__sub__`, or `__mul__`
- `__eq__`, `__ne__`, `__lt__`, `__gt__`, `__le__` or `__ge__`

You can practise implementing these dunder methods in the [Assignment]() notebook. We also recommend looking at the examples given in ***Fluent Python*** and trying them out for yourselves.

---
<a id='better'></a>
## Better integration - Leading underscore attributes

So far you've managed to build a solid class, but there a few things you can do that is considered more pythonic.

Currently you can see all attributes and methods available by instantiating deck, writing `deck.` and hitting `<tab>`.

Try this out in the cell below:

So to access the deck of cards outside the class itself is simple

In [ ]:
deck.cards[:5]

However accessing the cards variable outside the class is a little hacky and dangerous. You only want cards to be assigned when the class is first instantiated, or when deal is called... but a user could easily overwrite this attribute **outside** of the class

In [ ]:
deck.cards = 'my cards'
deck.cards

oops! 

<a id=leading></a>

### Leading underscore

To signal that an attribute should not be 'touched' externally to the class you should add a leading underscore - this indicates a **protected** variable.

In [ ]:
class Deck:
    ranks = '23456789TJQKA'
    suits = '♠♥♦♣'
    
    def __init__(self):
        self._cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]
    
    def deal(self):
        return self._cards.pop()

deck = Deck()

Now if you use `deck.<tab>` you see that you can no longer see the `cards` attribute.

Note that it is still possible to access the `_cards` attribute, and therefore it is possible to overwrite it. However the leading underscore is a clear signal to any user of the class that this attribute ***should not be accessed outside of the class*** and therefore great care should be taken to not overwrite the variable.

<a id=protected></a>

### Protecting variables further

For Python, using one leading underscore is considered enough to signal to the user that this attribute should not be used/overwritten.

**Unlike Java with the private modifier, there is no way to create private variables in Python**. In the notebook [Protect_Private_Variables](), further ways to protect variables with the use of Python's "private" variables, along with consideration around Pythonic practices, are discussed and demonstrated.

<img src=../images/conclusion.png align=right>
<a id=conclusion></a>

# Conclusion

In this tutorial you have used better practice when writing OOP in Python. 

In particular, this notebook has covered:

- Using dunder methods for consistent, Pythonic applications
- Using leading underscores for better integration